<a href="https://colab.research.google.com/github/xinle1030/RecipeTransformer/blob/Jon/recipe_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
import re
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import spacy
from nltk import chunk,pos_tag
# spacy.load('en')
from spacy.lang.en import English
parser = English()
nlp = spacy.load("en_core_web_sm")
import json
import re

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [59]:
# Get Synonyms for attribute fucntion 
def get_synonyms(word: str):
    synonyms = []
    for syn in wordnet.synsets(word):
        for i in syn.lemmas():
            synonyms.append(i.name())
    synonyms.append(word)
    return set(synonyms) 

def clean_synonyms(synonym_list, unused_synonyms):
  for synonym in unused_synonyms:
    if synonym in synonym_list:
      synonym_list.remove(synonym)
  ret_list = []
  for synonym in synonym_list:
    ret_list.append(synonym.lower())
  return ret_list

# get synonyms for attribute 
cook_time_synonyms = get_synonyms('cook')
serving_synonyms = get_synonyms('Serving')
prep_time_synonyms = get_synonyms('Prep')
ingredient_synonyms = get_synonyms('Ingredients')
instruction_synonyms = get_synonyms('Method')
tips_synonyms = get_synonyms('tips')
nutrient_synonyms = get_synonyms('Nutrition')
cuisine_synonyms = get_synonyms('Cuisine')
category_synonyms = get_synonyms('Categories')

# remove some synonyms from wordnet 
cook_time_synonyms = clean_synonyms(cook_time_synonyms, ['James_Cook', 'Captain_Cook', 'Captain_James_Cook', 'wangle', 'ready', 'falsify', 'make', 'Cook', 'prepare', 'misrepresent', 'fake', 'manipulate', 'fix', 'fudge'])
serving_synonyms = clean_synonyms(serving_synonyms, ['attend_to', 'suffice', 'dish_out', 'portion', 'helping', 'function', 'wait_on', 'answer', 'swear_out', 'attend', 'do', 'help', 'process', 'serving', 'dish'])
serving_synonyms.append("serves")
prep_time_synonyms = clean_synonyms(prep_time_synonyms, ['homework'])
ingredient_synonyms = clean_synonyms(ingredient_synonyms, ['fixings'])
instruction_synonyms = clean_synonyms(instruction_synonyms, ['method_acting'])
tips_synonyms = clean_synonyms(tips_synonyms, ['tippytoe', 'gratuity', 'lead', 'bakshish', 'steer', 'lean', 'summit', 'crest', 'backsheesh', 'tiptoe', 'baksheesh', 'confidential_information', 'hint', 'angle', 'slant', 'topple', 'tumble', 'top', 'wind', 'bakshis', 'crown', 'fee', 'tap', 'tip_off', 'tilt', 'bung', 'pourboire'])
nutrient_synonyms = clean_synonyms(nutrient_synonyms, ['victuals'])
cuisine_synonyms = clean_synonyms(cuisine_synonyms, [])
category_synonyms = clean_synonyms(category_synonyms, [])

# remove cook time first
list_of_attribute = [serving_synonyms, prep_time_synonyms, ingredient_synonyms, instruction_synonyms,
                     tips_synonyms, nutrient_synonyms, cuisine_synonyms, category_synonyms]

for attributes in list_of_attribute:
  print(attributes)

['service', 'dish_up', 'serving', 'service_of_process', 'serve', 'serve_well', 'assist', 'serve_up', 'serves']
['preparation', 'prep', 'prep']
['component', 'constituent', 'factor', 'ingredients', 'element', 'ingredient']
['method', 'method']
['tip', 'point', 'tips', 'peak']
['nutrition', 'sustenance', 'nutriment', 'nutrition', 'alimentation', 'nourishment', 'aliment']
['cuisine', 'cuisine', 'culinary_art']
['category', 'class', 'categories', 'family']


In [60]:
# Read input file 
from google.colab import files
uploaded = files.upload()


Saving page_3.txt to page_3.txt


In [61]:
# Read input file and store text from uploaded text file into "text" 
#from google.colab import files
#uploaded = files.upload()

#for fn in uploaded.keys():
#    text = uploaded.get(fn).decode(errors='replace')

In [62]:
#!unzip important_files.zip


# New Section

In [63]:
# Write to Synonym Json file

with open('important_files/synonyms.json', 'r') as f:
  json_object = json.load(f)
  for i, key in enumerate(json_object):
    json_object[key] = list(list_of_attribute[i])

with open('synonyms.json', 'w') as f:
    json.dump(json_object, f, indent=2)

In [64]:
# open text file in read mode
text_file = open('important_files/input_text.txt', 'rb')
text = text_file.read().decode(errors='replace')

# close files
text_file.close()


In [65]:
# Text Cleaning

def remove_non_ascii(text):
    return re.sub('[^-*.A-Za-z0-9,! ]+', '', text)

# make the line into a list  
clean_text_lines = []
flag = True
for line in text.splitlines():
    clean_text_lines.append(line.replace("\n", "").strip())   # remove newline character and space

In [66]:
with open('important_files/synonyms.json', 'r') as f:
  syn_json_object = json.load(f)

with open('important_files/recipe_template.json') as f:
  data = json.load(f)

with open('important_files/recipe_file.json', 'w') as f:
  json.dump(data, f, indent=2)

with open('important_files/recipe_file.json', 'r') as f:
  recipe_json_object = json.load(f)

result = []

for index, line in enumerate(clean_text_lines):
  for key in syn_json_object:
    # Division for Tips
    if key == "tips":
      if key in line.lower():
        result.append((key, index))
    # Division for Serving, Ingredient, Instructions, Nutrients
    else:
      for attribute in syn_json_object[key]:
        if attribute in line.lower() and key in recipe_json_object and not recipe_json_object[key]:
          result.append((key, index))
          recipe_json_object[key] = True

print(result)

[('serving', 0), ('ingredient', 3), ('tips', 11), ('instruction', 23), ('nutrient', 32)]


In [67]:
# Write to recipe json file

def categorize(json_obj, result):
    # general formatting 
    for i, item in enumerate(result):
      key = item[0]
      index = item[1]
      if key == "serving":
        serving_arr = re.findall(r'\d+', clean_text_lines[index])
        if len(serving_arr) > 0:
          serving = serving_arr[0]
          next_item = result[i + 1]
        else:
          serving = 0
          next_item = result[i]
        json_obj[key] = serving
        # title formatting        
        next_index = next_item[1]
        print(next_index)
        json_obj["title"] = clean_text_lines[index + 1 : next_index]
      elif key == "nutrient":
        last_index = len(clean_text_lines) - 3  # to exclude page label
        json_obj[key] = clean_text_lines[index + 1 : last_index]
      else:
        next_item = result[i + 1]
        next_index = next_item[1]
        json_obj[key] = clean_text_lines[index + 1 : next_index]
    return json_obj

def clear_text(text):
    return re.sub('[^-.A-Za-z0-9*,! ]+', '', text)

def format_steps(str_list):
  ret = []
  current = []
  for i in range(0, len(str_list)):
    string = str_list[i]
    if bool(re.match('^[\.a-zA-Z0-9,! ]*$', string[0])):
      current.append(string)
    else:
      step = " ".join(current)
      ret.append(clear_text(step.strip()))
      current = [string]
    if i == len(str_list) - 1:
      step = " ".join(current)
      ret.append(clear_text(step.strip()))
  if len(ret) > 0:
    ret.pop(0)
  return ret

def join_string(json_obj, key):
  # formatting title
  if key in json_obj:
    key_list = json_obj[key]
    if key == "instruction" or key == "tips" or key == "ingredient":
      json_obj[key] = format_steps(key_list)
    else:
      json_obj[key] = ' '.join(key_list)
  return json_obj

with open('important_files/recipe_template.json') as f:
    data = json.load(f)
    data = categorize(data, result)
    data = join_string(data, "title")
    data = join_string(data, "ingredient")
    data = join_string(data, "instruction")
    data = join_string(data, "tips")

with open('important_files/recipe_file.json', 'w') as f:
    json.dump(data, f, indent=2)
    print(data, '\n')
    for key in data:
      if key == "nutrient":
        print(key, ":", data[key])
        

3
{'id': '', 'breadcrumb': '', 'url': '', 'image': '', 'title': 'Stuffed Tomato with Garlic Sauce', 'serving': '4', 'category': '', 'cuisine': '', 'prep_time': 0, 'cook_time': 0, 'total_time': 0, 'ingredient': [' Tomato, big 200g', ' Frozen mixed vegetables 150g', ' Garlic, chopped 15g', ' Olive Oil 2 tsp', ' Cashew nut 20g', ' Oyster sauce 1 tbsp', ' Salt, to taste'], 'instruction': [' Halve tomatoes and remove seeds.', ' Heat Olive oil, add garlic and fry until fragrant.', ' Add frozen mixed vegetables, oyster sauce, cashew nuts and salt. Fry vegetables until cooked.', ' Fill tomatoes with stir-fried vegetables and bake in oven at 170C for 10 minutes.'], 'nutrient': ['Energy (1 kcal = 4.2kJ) 109kcal', 'Protein 3.06g', 'Total fat (g and % of total calories)', '5.3g (41.1%)', 'Saturated fat 0.9g', 'Cholesterol Omg', 'Carbohydrate 14.0g', 'Dietary Fibre 2.6g', 'Sodium 276mg'], 'tips': [' Grilling or baking, instead of deepfrying, is a great way to create tasty dishes that are healthier 

In [68]:
# Format nutrients element into properly formatted JSON object

with open('important_files/recipe_file.json', 'r') as f:
    i = 0
    result = {}
    json_object = json.load(f)
    nutrient = json_object["nutrient"]
    
    print(nutrient, '\n')
    while i < len(nutrient):
        searched_val = re.search(r"\d", nutrient[i])

        if searched_val: # element contains key and value (split and separate into key and value)
            index_firstnum = searched_val.start()
            if index_firstnum != 0: # both key and value
                key = nutrient[i][0:(index_firstnum-1)].strip() 
                value = nutrient[i][index_firstnum:len(nutrient[i])].strip()
                
                # Fix when bracket is in value instead of key
                if key[len(key)-1] == "(":
                    key = key[:len(key)]
                    right_brack_index = value.find(")")
                    key = key + value[:right_brack_index+1]
                    value = value[right_brack_index+1:len(value)].strip()
                i += 1
                
        else: # only key (make into key)
            if "Omg" in nutrient[i]: # Fixed issue where 0mg is read as Omg 
                split = nutrient[i].split(" ", 1)
                key = split[0]
                value = split[1]
                i += 1
            else:
                key = nutrient[i]
                value = nutrient[i + 1]
                i += 2
        result[key] = value               
    
    # Replace value of nutrient key with correctly formatted JSON 
    json_object["nutrient"] = result

with open('important_files/recipe_file.json', 'w') as f:
    json.dump(json_object, f, indent=2)


['Energy (1 kcal = 4.2kJ) 109kcal', 'Protein 3.06g', 'Total fat (g and % of total calories)', '5.3g (41.1%)', 'Saturated fat 0.9g', 'Cholesterol Omg', 'Carbohydrate 14.0g', 'Dietary Fibre 2.6g', 'Sodium 276mg'] 



In [69]:
# Nutrient